In [ ]:
#Power BI Dashboard Data Preparation


import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import json
import random
from geopy.geocoders import Nominatim
import warnings
warnings.filterwarnings('ignore')

class PowerBIDashboardPrep:
    """
    Prepare data specifically for Power BI dashboard visualization
    """
    
    def __init__(self, df_with_threats, model_results):
        self.df = df_with_threats
        self.model_results = model_results
        self.dashboard_data = {}
        
    def create_temporal_data(self, start_date="2024-01-01", days=30):
        """Create synthetic temporal data for real-time simulation"""
        print("📅 Creating temporal data for dashboard...")
        
        # Generate timestamps
        start = datetime.strptime(start_date, "%Y-%m-%d")
        timestamps = []
        
        for i in range(len(self.df)):
            # Random timestamp within the specified period
            random_days = random.uniform(0, days)
            random_hours = random.uniform(0, 24)
            timestamp = start + timedelta(days=random_days, hours=random_hours)
            timestamps.append(timestamp)
        
        self.df['Timestamp'] = timestamps
        self.df['Date'] = pd.to_datetime(self.df['Timestamp']).dt.date
        self.df['Hour'] = pd.to_datetime(self.df['Timestamp']).dt.hour
        self.df['Day_of_Week'] = pd.to_datetime(self.df['Timestamp']).dt.day_name()
        self.df['Week_Number'] = pd.to_datetime(self.df['Timestamp']).dt.isocalendar().week
        
        print(f"✅ Temporal data created from {min(timestamps)} to {max(timestamps)}")
        return self.df
    
    def create_geographic_data(self):
        """Create synthetic geographic data for IP mapping"""
        print("🌍 Creating geographic data...")
        
        # Sample cities and coordinates (in real scenario, use IP geolocation service)
        cities_data = [
            {"City": "New York", "Country": "USA", "Latitude": 40.7128, "Longitude": -74.0060},
            {"City": "London", "Country": "UK", "Latitude": 51.5074, "Longitude": -0.1278},
            {"City": "Tokyo", "Country": "Japan", "Latitude": 35.6762, "Longitude": 139.6503},
            {"City": "Sydney", "Country": "Australia", "Latitude": -33.8688, "Longitude": 151.2093},
            {"City": "Toronto", "Country": "Canada", "Latitude": 43.6532, "Longitude": -79.3832},
            {"City": "Berlin", "Country": "Germany", "Latitude": 52.5200, "Longitude": 13.4050},
            {"City": "Mumbai", "Country": "India", "Latitude": 19.0760, "Longitude": 72.8777},
            {"City": "São Paulo", "Country": "Brazil", "Latitude": -23.5505, "Longitude": -46.6333},
            {"City": "Moscow", "Country": "Russia", "Latitude": 55.7558, "Longitude": 37.6176},
            {"City": "Beijing", "Country": "China", "Latitude": 39.9042, "Longitude": 116.4074}
        ]
        
        # Assign random locations (weighted towards certain regions for attacks)
        locations = []
        for i in range(len(self.df)):
            if self.df.iloc[i]['Actual_Attack'] == 1:
                # Attacks more likely from certain regions
                weights = [0.05, 0.05, 0.05, 0.05, 0.05, 0.05, 0.1, 0.1, 0.25, 0.25]
            else:
                # Normal traffic more evenly distributed
                weights = [0.15, 0.15, 0.1, 0.1, 0.1, 0.1, 0.1, 0.05, 0.075, 0.075]
            
            location = random.choices(cities_data, weights=weights)[0]
            locations.append(location)
        
        # Add geographic columns
        geo_df = pd.DataFrame(locations)
        self.df['Source_City'] = geo_df['City']
        self.df['Source_Country'] = geo_df['Country']
        self.df['Source_Latitude'] = geo_df['Latitude']
        self.df['Source_Longitude'] = geo_df['Longitude']
        
        print(f"✅ Geographic data created for {len(cities_data)} locations")
        return self.df
    
    def create_network_metadata(self):
        """Create network-specific metadata for dashboard"""
        print("🌐 Creating network metadata...")
        
        # Protocol types
        protocols = ['TCP', 'UDP', 'ICMP', 'HTTP', 'HTTPS', 'FTP', 'SSH', 'DNS']
        protocol_weights = [0.3, 0.25, 0.1, 0.15, 0.1, 0.03, 0.04, 0.03]
        
        self.df['Protocol'] = random.choices(protocols, weights=protocol_weights, k=len(self.df))
        
        # Port numbers (simplified)
        common_ports = {
            'TCP': [80, 443, 21, 22, 23, 25, 53, 110, 143, 993, 995],
            'UDP': [53, 67, 68, 69, 123, 161, 162, 514],
            'HTTP': [80, 8080, 8000],
            'HTTPS': [443, 8443],
            'FTP': [21, 20],
            'SSH': [22],
            'DNS': [53],
            'ICMP': [0]
        }
        
        ports = []
        for protocol in self.df['Protocol']:
            if protocol in common_ports:
                port = random.choice(common_ports[protocol])
            else:
                port = random.randint(1024, 65535)
            ports.append(port)
        
        self.df['Destination_Port'] = ports
        
        # Service names
        service_mapping = {
            80: 'HTTP', 443: 'HTTPS', 21: 'FTP', 22: 'SSH', 
            23: 'Telnet', 25: 'SMTP', 53: 'DNS', 110: 'POP3'
        }
        
        self.df['Service'] = self.df['Destination_Port'].map(service_mapping).fillna('Other')
        
        print("✅ Network metadata created")
        return self.df
    
    def create_kpi_data(self):
        """Create KPI summary data for dashboard overview"""
        print("📊 Creating KPI data...")
        
        # Overall metrics
        total_flows = len(self.df)
        total_attacks = self.df['Actual_Attack'].sum()
        attack_rate = (total_attacks / total_flows) * 100
        avg_threat_score = self.df['Threat_Score'].mean()
        
        # Threat level distribution
        threat_distribution = self.df['Threat_Level'].value_counts().to_dict()
        
        # Top attack sources
        top_attack_countries = self.df[self.df['Actual_Attack'] == 1]['Source_Country'].value_counts().head(5).to_dict()
        
        # Protocol-wise attack rates
        protocol_attacks = self.df.groupby('Protocol').agg({
            'Actual_Attack': ['count', 'sum']
        }).round(2)
        protocol_attacks.columns = ['Total_Flows', 'Attack_Count']
        protocol_attacks['Attack_Rate'] = (protocol_attacks['Attack_Count'] / protocol_attacks['Total_Flows'] * 100).round(2)
        
        # Hourly patterns
        hourly_stats = self.df.groupby('Hour').agg({
            'Actual_Attack': ['count', 'sum'],
            'Threat_Score': 'mean'
        }).round(2)
        hourly_stats.columns = ['Total_Flows', 'Attack_Count', 'Avg_Threat_Score']
        
        # Daily patterns
        daily_stats = self.df.groupby('Day_of_Week').agg({
            'Actual_Attack': ['count', 'sum'],
            'Threat_Score': 'mean'
        }).round(2)
        daily_stats.columns = ['Total_Flows', 'Attack_Count', 'Avg_Threat_Score']
        
        kpi_data = {
            'overview': {
                'total_flows': int(total_flows),
                'total_attacks': int(total_attacks),
                'attack_rate': round(attack_rate, 2),
                'avg_threat_score': round(avg_threat_score, 2)
            },
            'threat_distribution': threat_distribution,
            'top_attack_countries': top_attack_countries,
            'protocol_stats': protocol_attacks.to_dict('index'),
            'hourly_patterns': hourly_stats.to_dict('index'),
            'daily_patterns': daily_stats.to_dict('index')
        }
        
        self.dashboard_data['kpis'] = kpi_data
        
        print("✅ KPI data created")
        return kpi_data
    
    def create_real_time_simulation_data(self):
        """Create data structure for real-time dashboard simulation"""
        print("⚡ Creating real-time simulation data...")
        
        # Sort by timestamp for streaming simulation
        df_sorted = self.df.sort_values('Timestamp').copy()
        
        # Create 5-minute intervals for streaming
        df_sorted['Time_Interval'] = pd.to_datetime(df_sorted['Timestamp']).dt.floor('5T')
        
        # Aggregate by intervals
        interval_stats = df_sorted.groupby('Time_Interval').agg({
            'Actual_Attack': ['count', 'sum'],
            'Threat_Score': ['mean', 'max'],
            'Source_Country': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown',
            'Protocol': lambda x: x.mode().iloc[0] if not x.mode().empty else 'Unknown'
        }).round(2)
        
        interval_stats.columns = ['Flow_Count', 'Attack_Count', 'Avg_Threat_Score', 'Max_Threat_Score', 'Top_Country', 'Top_Protocol']
        interval_stats['Attack_Rate'] = (interval_stats['Attack_Count'] / interval_stats['Flow_Count'] * 100).round(2)
        
        # Reset index to make timestamp a column
        interval_stats = interval_stats.reset_index()
        
        self.dashboard_data['real_time'] = interval_stats
        
        print(f"✅ Real-time simulation data created ({len(interval_stats)} intervals)")
        return interval_stats
    
    def create_threat_intelligence_data(self):
        """Create threat intelligence summary for dashboard"""
        print("🔍 Creating threat intelligence data...")
        
        # Attack type analysis (based on actual labels if available)
        if 'Label' in self.df.columns:
            attack_types = self.df[self.df['Actual_Attack'] == 1]['Label'].value_counts().head(10)
        else:
            # Create synthetic attack types based on threat scores
            synthetic_attacks = []
            for _, row in self.df[self.df['Actual_Attack'] == 1].iterrows():
                if row['Threat_Score'] > 80:
                    attack_type = random.choice(['DDoS', 'Brute Force', 'Infiltration'])
                elif row['Threat_Score'] > 60:
                    attack_type = random.choice(['Port Scan', 'Web Attack', 'Botnet'])
                else:
                    attack_type = random.choice(['DoS', 'Heartbleed', 'SQL Injection'])
                synthetic_attacks.append(attack_type)
            
            attack_types = pd.Series(synthetic_attacks).value_counts()
        
        # Severity classification
        severity_mapping = {
            'DDoS': 'Critical',
            'Infiltration': 'Critical',
            'Brute Force': 'High',
            'Botnet': 'High',
            'Web Attack': 'Medium',
            'Port Scan': 'Medium',
            'DoS': 'Medium',
            'Heartbleed': 'High',
            'SQL Injection': 'High'
        }
        
        # Create threat intelligence summary
        threat_intel = {}
        for attack_type, count in attack_types.items():
            threat_intel[attack_type] = {
                'count': int(count),
                'severity': severity_mapping.get(attack_type, 'Medium'),
                'percentage': round((count / len(self.df[self.df['Actual_Attack'] == 1])) * 100, 2)
            }
        
        self.dashboard_data['threat_intelligence'] = threat_intel
        
        print("✅ Threat intelligence data created")
        return threat_intel
    
    def export_for_powerbi(self, output_dir="powerbi_data/"):
        """Export all data in Power BI friendly formats"""
        print(f"📤 Exporting data for Power BI to {output_dir}...")
        
        import os
        os.makedirs(output_dir, exist_ok=True)
        
        # 1. Main dataset
        main_data = self.df[[
            'Timestamp', 'Date', 'Hour', 'Day_of_Week',
            'Source_City', 'Source_Country', 'Source_Latitude', 'Source_Longitude',
            'Protocol', 'Destination_Port', 'Service',
            'Threat_Score', 'Threat_Level', 'Actual_Attack'
        ]].copy()
        
        main_data.to_csv(f"{output_dir}network_traffic_data.csv", index=False)
        print("  ✅ Main dataset exported")
        
        # 2. KPI summary
        kpi_summary = pd.DataFrame([self.dashboard_data['kpis']['overview']])
        kpi_summary.to_csv(f"{output_dir}kpi_summary.csv", index=False)
        print("  ✅ KPI summary exported")
        
        # 3. Geographic summary
        geo_summary = self.df.groupby(['Source_Country', 'Source_City', 'Source_Latitude', 'Source_Longitude']).agg({
            'Actual_Attack': ['count', 'sum'],
            'Threat_Score': 'mean'
        }).round(2)
        geo_summary.columns = ['Total_Flows', 'Attack_Count', 'Avg_Threat_Score']
        geo_summary['Attack_Rate'] = (geo_summary['Attack_Count'] / geo_summary['Total_Flows'] * 100).round(2)
        geo_summary = geo_summary.reset_index()
        geo_summary.to_csv(f"{output_dir}geographic_summary.csv", index=False)
        print("  ✅ Geographic summary exported")
        
        # 4. Temporal patterns
        temporal_summary = self.dashboard_data['real_time']
        temporal_summary.to_csv(f"{output_dir}temporal_patterns.csv", index=False)
        print("  ✅ Temporal patterns exported")
        
        # 5. Protocol analysis
        protocol_summary = pd.DataFrame.from_dict(self.dashboard_data['kpis']['protocol_stats'], orient='index')
        protocol_summary = protocol_summary.reset_index()
        protocol_summary.rename(columns={'index': 'Protocol'}, inplace=True)
        protocol_summary.to_csv(f"{output_dir}protocol_analysis.csv", index=False)
        print("  ✅ Protocol analysis exported")
        
        # 6. Threat intelligence
        threat_intel_df = pd.DataFrame.from_dict(self.dashboard_data['threat_intelligence'], orient='index')
        threat_intel_df = threat_intel_df.reset_index()
        threat_intel_df.rename(columns={'index': 'Attack_Type'}, inplace=True)
        threat_intel_df.to_csv(f"{output_dir}threat_intelligence.csv", index=False)
        print("  ✅ Threat intelligence exported")
        
        # 7. Create DAX measures file (text file with DAX formulas)
        dax_measures = """
-- Key Performance Indicators
Total Traffic = COUNTROWS('network_traffic_data')

Total Attacks = SUMX('network_traffic_data', 'network_traffic_data'[Actual_Attack])

Attack Rate = DIVIDE([Total Attacks], [Total Traffic], 0) * 100

Average Threat Score = AVERAGE('network_traffic_data'[Threat_Score])

-- Time Intelligence
Current Hour Attacks = 
CALCULATE(
    [Total Attacks],
    FILTER(
        'network_traffic_data',
        'network_traffic_data'[Hour] = HOUR(NOW())
    )
)

Previous Hour Attacks = 
CALCULATE(
    [Total Attacks],
    FILTER(
        'network_traffic_data',
        'network_traffic_data'[Hour] = HOUR(NOW()) - 1
    )
)

Attack Trend = [Current Hour Attacks] - [Previous Hour Attacks]

-- Top Attackers (Dynamic)
Top Attack Country = 
TOPN(
    1,
    SUMMARIZE(
        FILTER('network_traffic_data', 'network_traffic_data'[Actual_Attack] = 1),
        'network_traffic_data'[Source_Country],
        "Attack Count", [Total Attacks]
    ),
    [Attack Count],
    DESC
)

-- Threat Level Analysis
Critical Threats = 
CALCULATE(
    [Total Traffic],
    'network_traffic_data'[Threat_Level] = "Critical"
)

High Threats = 
CALCULATE(
    [Total Traffic],
    'network_traffic_data'[Threat_Level] = "High"
)

Threat Level Distribution = 
CONCATENATEX(
    SUMMARIZE(
        'network_traffic_data',
        'network_traffic_data'[Threat_Level],
        "Count", [Total Traffic]
    ),
    'network_traffic_data'[Threat_Level] & ": " & [Count],
    ", "
)

-- Advanced Analytics
Rolling Average Threat Score = 
AVERAGEX(
    DATESINPERIOD(
        'network_traffic_data'[Date],
        LASTDATE('network_traffic_data'[Date]),
        -7,
        DAY
    ),
    [Average Threat Score]
)

Anomaly Score = 
IF(
    [Average Threat Score] > [Rolling Average Threat Score] * 1.5,
    "High Anomaly",
    IF(
        [Average Threat Score] > [Rolling Average Threat Score] * 1.2,
        "Medium Anomaly",
        "Normal"
    )
)

-- Geographic Analysis
Attack Density = 
DIVIDE(
    [Total Attacks],
    CALCULATE([Total Traffic], ALL('network_traffic_data'[Source_Country])),
    0
) * 100

-- Protocol Security Analysis
Protocol Risk Score = 
SWITCH(
    SELECTEDVALUE('network_traffic_data'[Protocol]),
    "TCP", [Attack Rate] * 1.2,
    "UDP", [Attack Rate] * 1.1,
    "ICMP", [Attack Rate] * 1.5,
    "HTTP", [Attack Rate] * 1.3,
    "HTTPS", [Attack Rate] * 0.8,
    [Attack Rate]
)
"""
        
        with open(f"{output_dir}dax_measures.txt", "w") as f:
            f.write(dax_measures)
        print("  ✅ DAX measures exported")
        
        # 8. Export model performance data
        if hasattr(self, 'model_results'):
            model_performance = []
            for model_name, results in self.model_results.items():
                if 'test_predictions' in results:
                    from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
                    
                    y_true = self.df['Actual_Attack']  # Assuming this exists
                    y_pred = results['test_predictions']
                    
                    if len(y_true) == len(y_pred):
                        model_performance.append({
                            'Model': model_name,
                            'Accuracy': accuracy_score(y_true, y_pred),
                            'Precision': precision_score(y_true, y_pred),
                            'Recall': recall_score(y_true, y_pred),
                            'F1_Score': f1_score(y_true, y_pred)
                        })
            
            if model_performance:
                model_df = pd.DataFrame(model_performance)
                model_df.to_csv(f"{output_dir}model_performance.csv", index=False)
                print("  ✅ Model performance exported")
        
        print(f"\n✅ All data exported to {output_dir}")
        print("📋 Files created:")
        files = [
            "network_traffic_data.csv - Main dataset",
            "kpi_summary.csv - Key performance indicators", 
            "geographic_summary.csv - Geographic analysis",
            "temporal_patterns.csv - Time-based patterns",
            "protocol_analysis.csv - Protocol security analysis",
            "threat_intelligence.csv - Threat type analysis",
            "model_performance.csv - ML model comparison",
            "dax_measures.txt - Power BI DAX formulas"
        ]
        
        for file in files:
            print(f"  • {file}")
    
    def create_dashboard_structure_guide(self):
        """Create a guide for Power BI dashboard structure"""
        print("\n📊 POWER BI DASHBOARD STRUCTURE GUIDE")
        print("=" * 60)
        
        dashboard_structure = {
            "Page 1: Executive Overview": {
                "purpose": "High-level KPIs and summary",
                "visuals": [
                    "Card visuals for Total Traffic, Total Attacks, Attack Rate, Avg Threat Score",
                    "Donut chart for Threat Level Distribution", 
                    "Line chart for Attack Trends over Time",
                    "Bar chart for Top Attack Countries",
                    "Gauge for Real-time Threat Score"
                ],
                "slicers": ["Date Range", "Threat Level"],
                "features": ["Auto-refresh", "Alerts for high threat scores"]
            },
            
            "Page 2: Real-Time Monitor": {
                "purpose": "Live network monitoring simulation",
                "visuals": [
                    "Line chart with streaming data simulation",
                    "Map visual showing attack origins", 
                    "Matrix showing recent attacks",
                    "Funnel chart for attack severity pipeline"
                ],
                "slicers": ["Time Window", "Protocol", "Country"],
                "features": ["Auto-refresh every 30 seconds", "Bookmarks for incident response"]
            },
            
            "Page 3: Threat Intelligence": {
                "purpose": "Detailed threat analysis",
                "visuals": [
                    "Treemap for Attack Types",
                    "Scatter plot for Threat Score vs Traffic Volume",
                    "Heatmap for Hour vs Day attack patterns",
                    "Waterfall chart for threat progression"
                ],
                "slicers": ["Attack Type", "Severity Level", "Time Period"],
                "features": ["Drill-through to detailed records", "Custom tooltips"]
            },
            
            "Page 4: Geographic Analysis": {
                "purpose": "Geographic threat distribution",
                "visuals": [
                    "Filled map for global attack distribution",
                    "Bar chart for country-wise attack rates",
                    "Table with country details",
                    "Flow map showing attack vectors"
                ],
                "slicers": ["Continent", "Threat Level"],
                "features": ["Custom map shapes", "Geographic drill-down"]
            },
            
            "Page 5: ML Model Performance": {
                "purpose": "Model evaluation and insights",
                "visuals": [
                    "Column chart comparing model metrics",
                    "Line chart for model performance over time",
                    "Key Influencers visual for attack predictors",
                    "Decomposition tree for threat factors"
                ],
                "slicers": ["Model Type", "Evaluation Metric"],
                "features": ["AI visuals", "What-if parameters"]
            }
        }
        
        print("📋 DASHBOARD PAGES STRUCTURE:")
        for page, details in dashboard_structure.items():
            print(f"\n🔹 {page}")
            print(f"   Purpose: {details['purpose']}")
            print(f"   Key Visuals: {', '.join(details['visuals'][:3])}...")
            print(f"   Interactive Elements: {', '.join(details['slicers'])}")
            print(f"   Advanced Features: {', '.join(details['features'])}")
        
        return dashboard_structure
    
    def run_complete_powerbi_prep(self):
        """Run complete Power BI preparation pipeline"""
        print("🚀 POWER BI PREPARATION PIPELINE")
        print("=" * 60)
        
        # Step 1: Create temporal data
        self.create_temporal_data()
        
        # Step 2: Create geographic data  
        self.create_geographic_data()
        
        # Step 3: Create network metadata
        self.create_network_metadata()
        
        # Step 4: Create KPI data
        self.create_kpi_data()
        
        # Step 5: Create real-time simulation data
        self.create_real_time_simulation_data()
        
        # Step 6: Create threat intelligence
        self.create_threat_intelligence_data()
        
        # Step 7: Export all data
        self.export_for_powerbi()
        
        # Step 8: Create dashboard guide
        self.create_dashboard_structure_guide()
        
        
        return True

def main():
    """Main function for Power BI preparation"""
    print("📊 Power BI Dashboard Preparation")
    print("=" * 60)
    
    
    # Load data 
    df_with_threats = pd.read_csv('test_data_with_threats.csv')
    
    # Load model results 
    import joblib
    model_results = joblib.load('models/model_results.pkl')
    
    # Initialize Power BI prep
    powerbi_prep = PowerBIDashboardPrep(df_with_threats, model_results)
    
    # Run complete preparation
    success = powerbi_prep.run_complete_powerbi_prep()
    
    if success:
        print("...")
    

if __name__ == "__main__":
    main()
            

📊 Power BI Dashboard Preparation
🚀 POWER BI PREPARATION PIPELINE
📅 Creating temporal data for dashboard...
✅ Temporal data created from 2024-01-01 00:37:28.921058 to 2024-01-31 23:08:57.777325
🌍 Creating geographic data...
✅ Geographic data created for 10 locations
🌐 Creating network metadata...
✅ Network metadata created
📊 Creating KPI data...
✅ KPI data created
⚡ Creating real-time simulation data...
✅ Real-time simulation data created (8479 intervals)
🔍 Creating threat intelligence data...
✅ Threat intelligence data created
📤 Exporting data for Power BI to powerbi_data/...
  ✅ Main dataset exported
  ✅ KPI summary exported
  ✅ Geographic summary exported
  ✅ Temporal patterns exported
  ✅ Protocol analysis exported
  ✅ Threat intelligence exported
  ✅ DAX measures exported
  ✅ Model performance exported

✅ All data exported to powerbi_data/
📋 Files created:
  • network_traffic_data.csv - Main dataset
  • kpi_summary.csv - Key performance indicators
  • geographic_summary.csv - Geogr